In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Creating a dataframe that has the following columns:
# 1. playID
# 2. nflID of tackler who made tackle for each playID
# 3. name of tackler who made tackle for each playID
# 4. Ball-carrier for each playID


#Import Data
games = pd.read_csv('../games.csv')
playerData = pd.read_csv('../players.csv')
tackleData = pd.read_csv('../tackles.csv')
tracking1 = pd.read_csv('../tracking_week_1data.csv')
plays = pd.read_csv("../plays.csv")

In [25]:
ball_carrier = pd.DataFrame(plays.loc[:, ['playId', 'ballCarrierId', 'gameId', 'down', 'yardsToGo', 'quarter', 'gameClock']])
ball_carrier['timeLeft'] = (4 - plays['quarter']) * 15
ball_carrier['scoreDif'] = plays['preSnapHomeScore'].sub(plays['preSnapVisitorScore'])
tackler = pd.DataFrame(tackleData.loc[:, ['playId', 'nflId', 'gameId']])
name = pd.DataFrame(playerData.loc[:, ['nflId', 'displayName']])

df = pd.merge(ball_carrier, tackler, on=('playId', 'gameId'))
df = pd.merge(df, name, on='nflId')

df.head(10)

,playId,ballCarrierId,gameId,down,yardsToGo,quarter,gameClock,timeLeft,scoreDif,nflId,displayName
0,3537,48723,2022100908,1,10,4,7:52,0,14,43306,Keanu Neal
1,84,47853,2022110609,2,10,1,14:50,45,0,43306,Keanu Neal
2,1016,53555,2022102301,1,10,2,11:29,30,0,43306,Keanu Neal
3,879,52460,2022110609,3,1,1,0:58,45,3,43306,Keanu Neal
4,97,46155,2022102700,1,10,1,14:18,45,0,43306,Keanu Neal
5,1557,54604,2022102700,2,9,2,4:18,30,7,43306,Keanu Neal
6,1410,47853,2022110609,1,10,2,9:21,30,-4,43306,Keanu Neal
7,1943,53453,2022101607,3,10,2,0:36,30,4,43306,Keanu Neal
8,1451,41325,2022100213,3,1,2,5:56,30,-11,43306,Keanu Neal
9,2400,47853,2022110609,1,10,3,11:22,15,-1,43306,Keanu Neal


In [27]:
print(plays['gameClock'].str[0:-3])

0         7
1         7
2         8
3        13
4         8
         ..
12481    13
12482     6
12483     9
12484     2
12485    12
Name: gameClock, Length: 12486, dtype: object


In [4]:
playerData = pd.read_csv("../players.csv")
tackleData = pd.read_csv("../tackles.csv")

tackles = {}

for row in range(len(playerData)):
    tackles[playerData.loc[row][0]] = [playerData.loc[row][0], 0, 0, 0, 0]


for row in range(len(tackleData)):
    if tackleData.loc[row][3] == 1:
        tackles[tackleData.loc[row][2]][1] = (tackles[tackleData.loc[row][2]][1] + 1)
    if tackleData.loc[row][4] == 1:
        tackles[tackleData.loc[row][2]][2] = (tackles[tackleData.loc[row][2]][2] + 1)
    if tackleData.loc[row][5] == 1:
        tackles[tackleData.loc[row][2]][3] = (tackles[tackleData.loc[row][2]][3] + 1)
    if tackleData.loc[row][6] == 1:
        tackles[tackleData.loc[row][2]][4] = (tackles[tackleData.loc[row][2]][4] + 1)
newTackles = pd.DataFrame(tackles)
newTackles = newTackles.transpose()
newTackles.rename(columns = {0:'nflId', 1:'Tackles', 2:'Assists', 3:'Forced Fumbles', 4:'Missed Tackles'}, inplace = True)
#newTackles.head()

merged = pd.merge(playerData, newTackles, on='nflId')
merged.head(100)

,nflId,height,weight,birthDate,collegeName,position,displayName,Tackles,Assists,Forced Fumbles,Missed Tackles
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady,0,0,0,0
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters,0,0,0,0
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers,0,0,0,0
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis,0,0,0,0
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
95,40024,6-2,211,1992-04-27,California,WR,Keenan Allen,0,0,0,0
96,40026,5-9,179,1990-11-19,Texas,WR,Marquise Goodwin,0,0,0,0
97,40030,6-3,327,1989-07-11,Georgia,NT,John Jenkins,2,6,0,0
98,40031,5-11,195,1991-02-09,Rutgers,FS,Logan Ryan,5,4,1,0


In [23]:
print(merged.loc[merged['displayName'] == "Keanu Neal"])

     nflId height  weight   birthDate collegeName position displayName  \
299  43306    6-1     216  1995-07-26     Florida       FS  Keanu Neal   

     Tackles  Assists  Forced Fumbles  Missed Tackles  
299       25       11               0               8  
